In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../..')))

import pandas as pd
import numpy as np
from geopy.distance import geodesic
from scripts.locator import Locator

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


### accessibility score: 
- distance to airport 
- distance to train stations
- altitude?

In [3]:
df_municipalities = pd.read_csv("../../data/processed/filtered_files/filtered_municipalities.csv")
df_airports = pd.read_csv("../../data/processed/filtered_files/filtered_airports.csv")
df_railway = pd.read_csv("../../data/processed/filtered_files/filtered_trains.csv")

In [4]:
df_municipalities.head()

,cmun,municipality,municipality_1,municipality_2,province,autonomous_community,latitude,longitude,altitude
0,1001,Alegría-Dulantzi,alegriadulantzi,NaN,Álava,País Vasco,42.84149,-2.513507,561.6857
1,1002,Amurrio,amurrio,NaN,Álava,País Vasco,43.05265,-3.001022,219.6910
2,1003,Aramaio,aramaio,NaN,Álava,País Vasco,43.05400,-2.566000,381.8797
3,1004,Artziniega,artziniega,NaN,Álava,País Vasco,43.12220,-3.128209,196.9808
4,1006,Armiñón,arminon,NaN,Álava,País Vasco,42.72305,-2.872574,463.5815


In [5]:
df_airports.head()

,airport,latitude,longitude,y_passengers,scaled_size
0,Adolfo Suárez Madrid - Barajas,40.492080,-3.576319,60220984,752762.3000
1,Albacete,38.949634,-1.863164,2644,33.0500
2,Alicante-Elche Miguel Hernández,38.285826,-0.563349,15747678,196845.9750
3,Almería,36.844781,-2.371234,775393,9692.4125
4,Asturias,43.562196,-6.032618,1974850,24685.6250


In [6]:
df_railway.sample(15)

,station_code,station_name,latitude,longitude,municipality,province
315,54403,El Chorro-Caminito Del Rey,36.907317,-4.759240,Álora,málaga
218,74205,Riglos-Concilio,42.316965,-0.727418,"Peñas De Riglos, Las",huesca
285,20218,Pobra De Brollon,42.533752,-7.400153,"Pobra Do Brollón, A",lugo
503,35005,Illescas,40.120872,-3.842335,Illescas,toledo
377,23004,Pontevedra,42.421640,-8.635830,Pontevedra,pontevedra
430,82100,Soria,41.755282,-2.475496,Soria,soria
389,23008,Vilagarcia De Arousa,42.601975,-8.762266,Vilagarcía De Arousa,pontevedra
524,69003,La Pobla Del Duc,38.915336,-0.445484,"Pobla Del Duc, La",valencia/valéncia
413,33013,Ciudad Rodrigo,40.608735,-6.530465,Ciudad Rodrigo,salamanca
489,67103,Villafranca Del Campo,40.684065,-1.357900,Villafranca Del Campo,teruel


In [7]:
# Apply the function to each row in the towns DataFrame and create two new columns
df_municipalities[['closest_distance_airport', 'nearest_airport']] = Locator(df_municipalities, df_airports, 'airport_name').result
df_municipalities[['closest_distance_train', 'nearest_train_station']]  = Locator(df_municipalities, df_railway, 'station_name').result

In [10]:
df_accessibility = df_municipalities[["cmun", "municipality_name", "province", "altitude", "longitude", "latitude", "nearest_airport", "closest_distance_airport", "nearest_train_station", "closest_distance_train"]]

In [11]:
# df_municipalities = pd.read_csv("../data/processed/filtered_municipalities.csv")
# df_accessibility = pd.read_csv("../../data/scored/accessibility.csv")
df_hospitals = pd.read_csv("../../data/processed/filtered_hospitals.csv")

In [ ]:
df_accessibility = df_accessibility[["cmun", "municipality_name", "province", "altitude", "longitude", "latitude", "nearest_airport", "closest_distance_airport", "nearest_train_station", "closest_distance_train"]]
df_hospitals_general = df_hospitals.query("type == 'General'")

# Apply the function to each row in the towns DataFrame and create two new columns
df_accessibility[['closest_distance_hospital', 'nearest_hospital']] = Locator(df_accessibility, df_hospitals_general, 'hospital_name').result

In [28]:
df_schools = pd.read_csv("../../data/processed/filtered_schools.csv")
df_distances = pd.read_csv("../../data/processed/processed_distances.csv")
df_schools

,school_id,public_private,type,name,latitude,longitude
0,1,Public,Colegio público de educación infantil y primar...,CPEIP Abarzuza Zumadia HLHIP,42.726290,-2.022176
1,2,Public,Escuela pública de música,Esc. Mús. Púb. Ablitas,41.975297,-1.640062
2,3,Public,Escuela infantil,EI Ablitas Mi Cole,41.974719,-1.641089
3,4,Public,Colegio de educación infantil y primaria,CPEIP Ablitas S. Babil HLHIP,41.974047,-1.641728
4,5,Public,Colegio de educación infantil y primaria,CPEIP Aibar G. V. Casamayor HLHIP,42.590753,-1.361802
...,...,...,...,...,...,...
26601,26602,Public,Aula de educación permanente de adultos,A.E.P.A. Herrera del Duque,39.168134,-5.048491
26602,26603,Public,Aula de educación permanente de adultos,A.E.P.A. Madrigalejo,39.135204,-5.626350
26603,26604,Public,Centro de educación permanente de adultos,C.E.P.A. Coria,39.987626,-6.538688
26604,26605,Public,Centro de educación permanente de adultos,C.E.P.A. Talayuela,39.982802,-5.607226


In [25]:
df_schools_filtered = df_schools.dropna(subset=['latitude', 'longitude'])
df_distances[['closest_distance_school', 'nearest_school']] = Locator(df_distances, df_schools_filtered, 'name').result

In [ ]:
# df_accessibility.to_csv("../../data/processed/processed_distances.csv", index=False)
df_distances.to_csv("../../data/processed/filtered_files/processed_distances.csv", index=False)